<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/Computaci%C3%B3n%20cu%C3%A1ntica%20adiab%C3%A1tica%20y%20Quantum%20Annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos qiskit en el notebook
!pip install qiskit qiskit-aer qiskit-algorithms qiskit-optimization qiskit_machine_learning qiskit-ibm-runtime pylatexenc

In [ ]:
import numpy as np
from math import sqrt

# importing Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator, StatevectorSimulator

# import basic plot tools
from qiskit.visualization import plot_histogram

# Funciones auxiliares

# Función para obtener y mostrar el vector de estado
def obten_estado(qcirc, etiqueta="|\psi\!\!> = ", bloch=False):
  estado = Statevector.from_instruction(qcirc)
  display(estado.draw('latex', prefix=etiqueta, max_size=2**qc.num_qubits))
  if bloch:
    display(estado.draw('bloch'))

# **Computación cuántica adiabática y Quantum Annealing**

### Contenidos

1. [Computación cuántica adiabática](#adiabatica)
1. [Quantum Annealing](#annealing)
1. [Discretización de la evolución adiabática (*Trotterization*)](#trotter)
1. [Algoritmos cuánticos híbridos y circuitos parametrizados](#hibrida)
1. [Quantum Approximate Optimization Algorithm (QAOA)](#qaoa)
1. [Variational Quantum Eigensolvers (VQE)](#vqe)


<a name="adiabatica"></a>

## **Computación cuántica adiabática**

Forma alternativa de computación cuántica basada en la evolución temporal de un Hamiltoniano y en el [teorema adiabático](https://en.wikipedia.org/wiki/Adiabatic_theorem)

#### Ecuación de Schrödinger

La evolución temporal de un sistema cuántico viene dada por la ecuación de Schrödinger:

$$i\hbar \frac{\partial}{\partial t}|\psi(t)\rangle = H(t)|\psi(t)\rangle$$

$H(t)$ es el Hamiltoniano del sistema en el instante $t$ y define la energía del mismo.

#### Teorema adiabático
Si un sistema cuántico está en un estado base (estado de menor energía o _ground state_) de un Hamiltoniano inicial y se le hace evolucionar muy lentamente hacia un Hamiltoniano final, este estado inicial evolucionará al estado base del Hamiltoniano final, tras un tiempo suficientemente largo.

Supongamos que tenemos un Hamiltoniano $H_c$ (_Hamiltoniano de coste_) cuyo estado base codifica la solución de nuestro problema.

Partimos de un Hamiltoniano $H_0$ cuyo estado base es facil de preparar y queremos llegar, en un tiempo $T$, al estado base de $H_c$. Hacemos evolucionar el sistema desde $t=0$ hasta $t=T$ como sigue:

$$H(t) = A(t)H_0 + B(t)H_c$$

donde las funciones $A$ y $B$ verifican: $A(0) = B(T) = 1$ y $A(T) = B(0) = 0$.

Un ejemplo de estas funciones son

$$
A(t) =  \left(1-\frac{t}{T}\right)\\
B(t) = \frac{t}{T}
$$


Si $t$ cambia muy lentamente, el sistema pasa de $H(0) = H_0$ a $H(T) = H_c$ manteniéndose en el estado base.

Midiendo el estado del sistema en $t=T$ obtenemos el estado $|\psi\rangle$ que minimiza $\langle\psi|H_c|\psi\rangle$ y que es la solución del problema.

En [Aharonov et al. 2007](https://doi.org/10.1137/080734479) se demostró que la computación cuántica adiabática es equivalente a la computación cuántica basada en puertas.

#### Tiempo de evolución

El teorema adiabático establece que para que $H(t)$ se mantenga en el estado base, $T$ tiene que ser inversamente proporcional al cuadrado del *spectral gap* (minima diferencia de energía entre el estado base y el primer estado excitado de $H(t)$ con $t\in[0,T]$).

Encontrar el valor del *spectral gap* es muy complejo. Además, $T$ podría llegar a ser muy grande. El *Quantum annealing* es una implementación práctica de la computación cuántica adiabática.




<a name="annealing"></a>

## **Quantum annealing**

Es un modelo similar a la computación cuántica adiabática, con ciertas limitaciones:

- El Hamiltoniano final tiene que ser un Hamiltoniano Ising:  

$$
H_c = -\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - \sum_{i=0}^{n-1} h_{i}Z_i
$$

- No se mantiene la evolución adiabática $\implies$ el estado final podría no ser el estado base de $H_c$

**Estado inicial $H_0$**

Se suele usar como estado inicial el siguiente:

$$
H_0 = -\sum_{i=0}^{n-1} X_i
$$

Este estado se denomina _mixing Hamiltonian_ y su estado base es $|\psi_0\rangle = |+\rangle^{\otimes n}$

**Evolución del Hamiltoniano**

La evolución viene dada por:

$$
H(t) = A(t)H_0 + B(t)H_c = -A(t)\sum_{i=0}^{n-1} X_i - B(t)\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - B(t)\sum_{i=0}^{n-1} h_{i}Z_i
$$


### Quantum annealers
Computadores cuánticos que resuelven problemas de optimización mediante Quantum annealing.

- [D-Wave Systems](https://www.dwavesys.com/): Primera empresa en vender quantum annealers
  - El último modelo dispone de más de 7000 cúbits
  - Suite [Ocean](https://docs.ocean.dwavesys.com/en/latest/)
  - Tipo de problemas: optimización de portfolios, optimización de rutas y gestión de flotas, optimización de líneas de producción, búsqueda de similaridad entre moléculas, etc.




In [ ]:
!pip install dwave-neal dimod

In [ ]:
# dimod: biblioteca base de D-Wave Ocean para representar
# y resolver problemas combinatorios binarios cuadráticos
from dimod import BinaryQuadraticModel, SPIN
J = {(0,1):1, (0,2):1}
h = {}

# Expresamos el grafo simple como un modelo binario cuadrático
# SPIN: las variables toman valores {-1,1}
problem = BinaryQuadraticModel(h, J, 0.0, SPIN)
print("The problem we are going to solve is:")
print(problem)

In [ ]:
# neal: simulador de annealing de DWave
from neal import SimulatedAnnealingSampler

# Usamos un simulador clásico en vez del D-WaveSampler
sampler = SimulatedAnnealingSampler()

# Ejecutamos la muestra con 10 repeticiones
result = sampler.sample(problem, num_reads=10)

print("\nThe solutions that we have obtained are:")
print(result)


In [ ]:
# Uso en un sistemna Dwave
# 1. Instalar el SDK de Ocean con pip install dwave-ocean-sdk
# 2. Registrarse en Leap (https://cloud.dwavesys.com/leap/) y obtener una clave API
# 3. Usar "dwave config create" para añadir la clave a nuestra configuración
#
# Código adaptado
#from dwave.system import DWaveSampler
#from dwave.system import EmbeddingComposite
#sampler = EmbeddingComposite(DWaveSampler())
#result = sampler.sample(problem, num_reads=10)
#print("The solutions that we have obtained are")
#print(result)



---

---

<a name="trotter"></a>
## **Discretización de la evolución adiabática (*Trotterization*)**

Podemos replicar la evolución adiabática en un computador cuántico basado en puertas discretizando la evolución continua del Hamiltoniano.

La evolución temporal de un sistema viene dada por la ecuación de Schrödinger:

$$i\hbar \frac{d}{dt}|\psi(t)\rangle = H(t)|\psi(t)\rangle$$

En general, $H(t)$ depende del tiempo. Pero en un intervalo temporal $[t_i, t_i+\Delta t]$ suficientemente pequeño, podemos considerar $H(t_i)$ constante, verificándose que:

$$|\psi(t_i+\Delta t)\rangle = e^{-iH(t_i)\Delta t}|\psi(t_i)\rangle = U_i|\psi(t_i)\rangle$$

siendo $U_i = e^{-iH(t_i)\Delta t}$ una matriz unitaria.



Si discretizamos la evolución en $p = \frac{T}{\Delta t}$ pasos, podemos escribir:

$$
|\psi(t_1)\rangle = U_0|\psi(t_0)\rangle \\
|\psi(t_2)\rangle = U_1|\psi(t_1)\rangle \\
\cdots \\
|\psi(t_{p})\rangle = U_{p-1}|\psi(t_{p-1})\rangle
$$


De esta forma, el estado final del sistema será:

$$
|\psi(t_{p})\rangle = U_{p-1}U_{p-2}\cdots U_{1}U_{0}|\psi(t_0)\rangle = \left(\prod_{m=p-1}^{0} U_m\right) |\psi(t_0)\rangle =
$$

El Hamiltoniano usado en computación adiabática es:

$$H(t) = A(t)H_0 + B(t)H_c$$

por lo que resulta:

$$
|\psi(T)\rangle = \left(\prod_{m=0}^{p-1} e^{i\Delta t(A'(t_m)H_0 + B'(t_m)H_c)}\right) |\psi(0)\rangle
$$

donde $A'(t_m) = -A(t_{p-1-m})$, $B'(t_m) = -B(t_{p-1-m})$, $|\psi(T)\rangle = |\psi(t_{p})\rangle$ y $|\psi(0)\rangle = |\psi(t_{0})\rangle$

En general, dadas dos matrices $A$ y $B$, la igualdad $e^{A+B} = e^Ae^B$ solo se da si $A$ y $B$ _conmutan_, es decir si $[A,B] = AB-BA = 0$.

Si $\Delta t$ es lo suficientemente pequeño, la formula de Lie-Trotter permite escribir:

$$
e^{i\Delta t(A'(t_m)H_0 + B'(t_m)H_c)} \approx e^{i\Delta tA'(t_m)H_0} e^{i\Delta tB'(t_m)H_c}
$$

Por lo tanto, el estado final del sistema se puede escribir:

$$
|\psi_f\rangle = |\psi(T)\rangle = \left(\prod_{m=0}^{p-1} e^{i\Delta tA'(t_m)H_0} e^{i\Delta tB'(t_m)H_c}\right) |\psi(0)\rangle =
\prod_{m=0}^{p-1} U_0(t_m)U_c(t_m)|\psi_0\rangle
$$

siendo las matrices unitarias:

$$
U_0(t_m) = e^{i\Delta tA'(t_m)H_0}\\
U_c(t_m) = e^{i\Delta tB'(t_m)H_c}
$$

Este procedimiento se denomina _trotterization_ y es la base de algoritmos como el QAOA.




---



---




<a name="hibrida"></a>
# **Algoritmos cuánticos híbridos y circuitos parametrizados**

Problema de los computadores cuánticos actuales: el ruído destruye la superposición de estado.

No es posible resolver un problema complejo (profundo) en un sistema actual: por ejemplo, la factorización de Shor podría necesitar millones de cúbits para corrección de errores.

Computadores actuales: _Noisy intermediate-scale quantum_ (NISQ)

Podemos pensar en dividir un problema complejo en partes más simples, y utilizar CPUs tradicionales en el proceso de optimización:

<center><img src="https://drive.google.com/uc?export=view&id=1WgpYw-cCkpuNZE2AOnPL4b3kIt9A_kU2" alt="Algoritmo híbrido" width="700"  /></center>

Esta estrategia también se denomina _Computación variacional_.

Se basa en usar un circuito cuántico que incluye parámetros que se van optimizando. Este circuitos se  denomina *ansatz*, _forma (o circuito) variacional_ o, simplemente, circuito cuántico parametrizado.

### Circuitos parametrizados en Qiskit

Qiskit incluye métodos para facilitar la creación de circuitos parametrizados.

In [ ]:
from qiskit.circuit import Parameter, ParameterVector

#Se crean los parámetros usando un string como identificador (no se les asignan ningún valor)
parameter_0 = Parameter('θ')
parameter_1 = Parameter('β')
circuit = QuantumCircuit(1)

#Podemos ahora usar los parámetros como ángulos de rotación en una puerta
circuit.ry(theta = parameter_0, qubit = 0)
circuit.rx(theta = parameter_1, qubit = 0)

circuit.draw('mpl')

Es posible definir vectores de parámetros

In [ ]:
# Numero de cubits y de parametros
n = 2
num_par = 2*n

# ParameterVectors se inicializan con un identificador y un entero que indica la longitud del string
parameters = ParameterVector('θ', num_par)

circuit = QuantumCircuit(n)

for i in range(n):
    circuit.ry(parameters[i], i)
    circuit.rx(parameters[i+n], i)

circuit.draw('mpl')

Finalmente, se dan valores a los parámetros.

In [ ]:
# Creamos un diccionario que asigna a cada parámetro un valor aleatorio
param_dict = {p: np.random.random() for p in parameters}
print(param_dict)

# El método assign_parameters permite ligar los valores al circuito
circuit_v = circuit.assign_parameters(parameters = param_dict)
circuit_v.draw('mpl')

-------------------

## Ejercicio 5.1

Crea el siguiente circuito parametrizado, que alterna capas de rotaciones $R_y$ con CNOTS. Esta forma variacional se denomina [RealAmplitudes](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RealAmplitudes.html) y se usa en química computacional y en problemas de clasificación en quantum machine learning.

<center><img src="https://drive.google.com/uc?export=view&id=1HKVx-jEzWhNPH-VNEpzATRyv0xG0fFfZ" alt="Ansatz Real Amplitudes" width="700"  /></center>


In [ ]:
from qiskit.circuit import Parameter, ParameterVector

# Número de cubits y capas Ry+CNOT (no cuenta la capa Ry final)
n = 4
num_layers = 2

# Crea el vector de parámetros
parameters = ParameterVector('t', n*(num_layers+1))

# Crea el circuito
circuit = QuantumCircuit(n)

# Crea las capas Ry+CNOT
for layer in range(num_layers):
    #
    # TODO: Añade las puertas Ry parametrizadas usando el vector de parámetros para obtener el estado 𝜓0
    #
    for i in range(n):
      circuit.ry(parameters[i+layer*n],i)


    # Barrera para visualización
    circuit.barrier()

    #
    # TODO: Añade las puertas CNOT
    #
    for i in range(n-1,0,-1):
      circuit.cx(i-1,i)


    # Barrera para visualización
    circuit.barrier()

#
# TODO: Añade la capa Ry final
#
for i in range(n):
  circuit.ry(parameters[i+(layer+1)*n],i)

circuit.draw('mpl')

In [ ]:
# Este circuito está implementado en la librería de Qiskit
from qiskit.circuit.library import RealAmplitudes
n = 4
num_layers = 2
ansatz = RealAmplitudes(n, reps=num_layers, insert_barriers=True)
ansatz.decompose().draw('mpl')



---



---



---



<a name="qaoa"></a>
# **Quantum Approximate Optimization Algorithm (QAOA)**

Algoritmo híbrido basado en la discretización de la evolución adiabática para resolver problemas Ising.

Como ya hemos visto, el estado final del sistema despues de la evolución es:

$$
|\psi_f\rangle = |\psi(T)\rangle = \left(\prod_{m=0}^{p-1} e^{i\Delta tA'(t_m)H_0} e^{i\Delta tB'(t_m)H_c}\right) |\psi(0)\rangle =
\prod_{m=0}^{p-1} U_0(t_m)U_c(t_m)|\psi_0\rangle
$$

siendo las matrices unitarias:

$$
U_0(t_m) = e^{i\Delta tA'(t_m)H_0} = e^{i\beta_mH_0} = U_0(\beta_m)\\
U_c(t_m) = e^{i\Delta tB'(t_m)H_c} = e^{i\gamma_mH_c} = U_c(\gamma_m)
$$
con $m=0,\ldots,p-1$ y $p \ge 1$.

QAOA intenta elegir unos valores de $\beta_m$ y $\gamma_m$ que aproximen el estado $|\psi_f\rangle$ al estado base del Hamiltoniano de coste $H_c$, es decir, que minimizen $\langle\psi_f|H_c|\psi_f\rangle$.

El algoritmo QAOA parte de un Hamiltoniano de coste $H_c$ que define el problema y procede como sigue:

1. Se elige un valor de $p\ge 1$ y dos listas de valores $\boldsymbol{\beta} = (\beta_0,\ldots,\beta_{p-1})$ y $\boldsymbol{\gamma} = (\gamma_0,\ldots,\gamma_{p-1})$
1. Como $H_0$ se suele usar el _mixing Hamiltonian_ y como estado inicial se suele usar la superposición completa
  $$H_0 = -\sum_{i=0}^{n-1} X_i\quad
  |\psi_0\rangle = |+\rangle^{\otimes n}$$
2. La QPU aplica las puertas $U_0(\beta_m)U_c(\gamma_m)$, $m=0,\ldots,p-1$, y se obtiene un nuevo estado $|\psi_p\rangle$
3. En la CPU se usa un algoritmo de optimización para actualizar los parámetros $\boldsymbol{\beta}$ y $\boldsymbol{\gamma}$ intentando minimizar el valor esperado $\langle\psi_p|H_c|\psi_p\rangle$
4. Se vuelve al paso 2 hasta obtener los valores óptimos $\boldsymbol{\beta}^\ast$ y $\boldsymbol{\gamma}^\ast$
5. Usando esos valores, se obtiene el estado que minimiza la solución

El proceso de optimización de los parámetros puede ser un simple gradiente descendente, en el que cada parámetro se actualiza en la dirección que conduzca al mayor descenso del valor esperado, u otros método de optimización más sofisticados.

### Construcción de $U_0(\beta)$ y $U_c(\gamma)$

#### Puertas para $U_0(\beta)$

Si tenemos $n$ cúbits, y dado que las matrices $X$ conmutan tenemos:

$$U_0(\beta) = e^{i\beta H_0} = e^{-i\beta\sum_{i=0}^{n-1} X_i} = \prod_{i=0}^{n-1}e^{-i\beta X_i}$$

Y recordando que la puerta de rotación $R_x(\theta) = e^{-i\theta X/2}$, se tiene:

$$U_0(\beta) = \prod_{i=0}^{n-1} R_x^{(i)}(2\beta)$$

La puerta $R_x^{(i)}$ indica que se aplica una $R_x$ al cúbit $i$ y la identidad al resto $\implies$ el producto equivale a aplicar una puerta $R_x(2\beta)$ a cada uno de los cúbits.

#### Puertas para $U_c(\gamma)$

El Hamiltoniano de coste de un problema Ising es:


$$H_c = -\sum_{i,j = 0}^{n-1} J_{ij}Z_iZ_j -\sum_{i = 0}^{n-1} h_i Z_i$$

siendo $J_{ij}$ y $h_i$ números reales.

Ya que las matricez $Z$ conmutan, podemos escribir

$$
U_c(\gamma) = e^{i\gamma H_c} = e^{-i\gamma(\sum_{i,j = 0}^{n-1} J_{ij}Z_iZ_j +\sum_{i = 0}^{n-1} h_i Z_i)} =
\prod_{i,j = 0}^{n-1}e^{-i\gamma J_{ij}Z_iZ_j}\prod_{i=0}^{n-1}e^{-i\gamma h_i Z_i}
$$

El segundo término se puede implementar con puertas $R_z(\theta) = e^{-i\theta Z/2}$.


Para el primer término, $Z_iZ_j$ representa aplicar una puerta $Z$ a los cúbits $i$ y $j$ y la identidad al resto:

$$
Z_iZ_j = I\otimes\ldots\otimes Z\otimes\ldots\otimes Z \otimes\ldots I = Z_i\otimes Z_j
$$

Y el término $e^{-i\gamma J_{ij}Z_i\otimes Z_j}$ corresponde a una [puerta Ising ZZ](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RZZGate.html#qiskit.circuit.library.RZZGate) entre los cúbits $i$ y $j$:

$$
R_{ZZ}(\theta) = \exp\left(-i\frac{\theta}{2}(Z\otimes Z)\right) =
\exp{\left(-i\frac{\theta}{2}\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix} \right)}=
\begin{bmatrix}
e^{-i\theta/2} & 0 & 0 & 0\\
0 & e^{i\theta/2} & 0 & 0\\
0 & 0 & e^{i\theta/2} & 0\\
0 & 0 & 0 & e^{-i\theta/2}\\
\end{bmatrix}
$$

Con lo que queda:

$$
U_c(\gamma) = \prod_{i,j = 0}^{n-1} R_{ZZ}^{(i,j)}(2\gamma J_{ij})\prod_{i=0}^{n-1}R_z^{(i)}(2\gamma h_i)
$$


In [ ]:
from qiskit.circuit import Parameter

# Puerta ZZ en Qiskit
theta = Parameter('θ')
qc = QuantumCircuit(2)

qc.rzz(theta, 0, 1)

display(qc.draw('mpl'))

display(qc.decompose().draw('mpl'))

### Ejemplo

Obtener el circuito QAOA para un paso ($p$=1) del Hamiltoniano Ising:

$$
H_c = 3Z_2Z_0 - Z_2Z_1 + 2Z_0
$$

Necesitamos 3 cúbits, y el estado que queremos conseguir será:

$$
U_0(\beta)U_c(\gamma) |+\rangle^{\otimes 3}
$$

Con

$$
U_0(\beta) = \prod_{i=0}^{2} R_x^{(i)}(2\beta) = R_x^{(0)}(2\beta)\otimes R_x^{(1)}(2\beta)\otimes R_x^{(2)}(2\beta)
$$

$$
U_c(\gamma) = \prod_{i,j = 0}^{2} R_{ZZ}^{(i,j)}(2\gamma J_{ij})\prod_{i=0}^{2}R_z^{(i)}(2\gamma h_i) = (R_{ZZ}^{(2,0)}(6\gamma)\otimes R_{ZZ}^{(2,1)}(-2\gamma))(R_z^{(0)}(4\gamma))
$$

El circuito sería:

In [ ]:
from qiskit.circuit import Parameter

n = 3

beta = Parameter('β')
gamma = Parameter('γ')

qc = QuantumCircuit(n)

# Estado superpuesto
qc.h(range(3))

# Puertas U_c
qc.rzz(6*gamma, 2, 0)
qc.rzz(-2*gamma, 2, 1)
qc.rz(4*gamma, 0)

# puertas U_0
for i in range(3):
    qc.rx(2*beta, i)

qc.draw('mpl')

Podemos usar la implementación de QAOA de Qiskit para obtener este mismo circuito

In [ ]:
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp

# Expresamos el problema como un operador Hamiltoniano
Hc = SparsePauliOp.from_list([("ZIZ", 3), ("ZZI", -1), ("IIZ", 2)])

# Creamos el ansatz QAOA
ansatz = QAOAAnsatz(Hc, reps=1)

ansatz.decompose(reps=2).draw("mpl")

La puerta $U_2$ es una rotación de 1 cúbit generalizada, de la forma:

$$
U_2(\phi, \lambda) = \frac{1}{\sqrt{2}}
\begin{pmatrix}
1          & -e^{i\lambda} \\
e^{i\phi} & e^{i(\phi + \lambda)}
\end{pmatrix}
$$

Así, una puerta $H$ es equivalente a:

$$
H = U_2(0, \pi) = \frac{1}{\sqrt{2}}
\begin{pmatrix}
1          & -e^{i\pi} \\
e^{i0} & e^{i(0 + \pi)}
\end{pmatrix} =
\frac{1}{\sqrt{2}}\begin{pmatrix}
1          & 1 \\
1 & -1
\end{pmatrix}
$$

Dado este Hamiltoniano, podemos usar fuerza bruta para encontrar los valores esperados para todos los estados base:

In [ ]:
from qiskit.quantum_info import Statevector

# Creamos una lista con todos los estados base para n cúbits
estados = [Statevector.from_int(i, dims=2**n) for i in range(2**n)]

for i in range(2**n):
  print('Estado ',i, 'valor esperado=',estados[i].expectation_value(Hc).real)

Usamos la implementación de [QAOA](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.QAOA.html) del paquete [qiskit_algorithms](https://qiskit-community.github.io/qiskit-algorithms/apidocs/qiskit_algorithms.html) para obtener el mínimo

In [ ]:
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit.result import QuasiDistribution

# Sampler: nueva primitiva de Qiskit que obtiene
# muestras del circuito para estimar el valor esperado.
sampler = Sampler()

algorithm_globals.random_seed = 10598

optimizer = COBYLA()

qaoa = QAOA(sampler, optimizer)

# Obtiene el mínimo autovalor del Hamiltoniano
result = qaoa.compute_minimum_eigenvalue(Hc)

print(result,'\n')

bistring = result.best_measurement['bitstring']
valor = result.best_measurement['value']
print("Estado que minimiza Hc = {}".format(bistring))
print("Valor esperado <{}|Hc|{}> = {}".format(bistring,bistring,valor.real))

# Nota: el warning se debe a que QAOA no está migrado a la V2 de Sampler

### Programas cuadráticos con restricciones cuadráticas

La librería de optimización de Qiskit incorpora una librería para resolver problemas QCQP con variables binarias y/o enteras.

**Ejemplo**

Minimiza $y=10x_0+20x_1+30 x_2 +  4 x_0x_1 + 2 x_0x_2 + 6 x_1x_2 + 4 x_1^2  + 2 x_2^2$

con las siguientes restricciones:

  - $x_0\in\{0,1\}$
  - $x_1, x_2 \in \mathbb{Z}, -1\le x_1 \le 1, -2\le x_2 \le 3$

Podemos expresar el problema de la siguiente forma:ç

Se puede escribir como:
$$
\begin{align}
\text{minimizar} &&x^T Q x + c^T x &&\\
&& && \\
\text{sujeto a} &&x_0\in\{0,1\}, x_1, x_2 \in \mathbb{Z}  &&\\
&& -1\le x_1 \le 1 \\
&& -2\le x_2 \le 3 \\
\end{align}
$$

Se puede probar fácilmente que $Q$ y $c$ en este ejemplo valen:

$$
\begin{aligned}
Q  &= \begin{bmatrix}0 & 1 & 2 \\ 3 & 4 & 5 \\ 0 & 1 & 2 \end{bmatrix}\\[10pt]
c^T &= \begin{bmatrix}10&20&30\end{bmatrix}
\end{aligned}
$$

In [ ]:
from qiskit_optimization import QuadraticProgram

# Definimos el problema cuadrático
qprog = QuadraticProgram('Ejemplo')

# Fijamos el tipo de las variables y sus restricciones
qprog.binary_var(name = 'x_0')
qprog.integer_var(name = 'x_1', lowerbound = -1, upperbound = 1)
qprog.integer_var(name = 'x_2', lowerbound = -2, upperbound = 3)

# Matriz Q y vector c
Q = [[0,1,2],[3,4,5],[0,1,2]]
c = [10,20,30]

# Exprersamos el problema como una minimización
qprog.minimize(quadratic = Q, linear = c)

print(qprog.prettyprint())

Este problema puede ser resuelto mediante QAOA usando el método [`MinimumEigenOptimizer`](https://qiskit-community.github.io/qiskit-optimization/stubs/qiskit_optimization.algorithms.MinimumEigenOptimizer.html) que se encarga de convertir el programa cuadrático a un circuito y de realizar la optimización.

A este método hay que pasarle un objeto de una clase que implemente la interfaz [`MinimumEigenSolver`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.MinimumEigensolver.html), como puede ser un objeto [`VQE`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.VQE.html) o [`QAOA`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.QAOA.html).

In [ ]:
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import ADAM
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer

qaoa = QAOA(Sampler(), ADAM(), initial_point=[0.0, 0.0])

# Creamos el optimizador
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa)

# Usamos ese optimizador sobre el problema anterior
result = eigen_optimizer.solve(qprog)

print(result)

Podemos comprobar que el resultado es correcto usando un minimizador clásico.

In [ ]:
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

np_solver = NumPyMinimumEigensolver()
np_optimizer = MinimumEigenOptimizer(min_eigen_solver = np_solver)

result = np_optimizer.solve(qprog)
print(result)

### Maxcut como QUBO

Ya vimos que el MAX-CUT se puede expresar como un problema QUBO, con una función de coste:
$$
C(\textbf{x}) = \sum_{i,j = 0}^{n-1} w_{ij} x_i (1-x_j)
$$
siendo todas las $x_i$ variables binarias.

Esta función de coste se puede reescribir como:

$$
\begin{align}
\sum_{i,j=0}^{n-1} w_{ij} x_i (1-x_j) &= \sum_{i,j=0}^{n-1} w_{ij} x_i - w_{ij}x_i x_j  \\
&= \sum_{i=0}^{n-1} \left( \sum_{j=0}^{n-1} w_{ij} \right) x_i - \sum_{i,j = 0}^{n-1} w_{ij}x_i x_j \\
&= c^T x + x^T Q x, \\
\end{align}
$$
siendo $Q$ y $c$:
$$
Q_{ij} = -w_{ij} \qquad c_i = \sum_{j=1}^n w_{ij}.
$$

In [ ]:
import networkx as nx  # Librería para manejar grafos

# Ejemplo de grafo con 6 nodos
nnodes = 6
G = nx.Graph()
# Añade nodos y aristas
G.add_nodes_from(np.arange(0,nnodes,1))
edges = [(0,1,2.0),(0,2,3.0),(0,3,2.0),(0,4,4.0),(0,5,1.0),(1,2,4.0),(1,3,1.0),
         (1,4,1.0),(1,5,3.0),(2,4,2.0),(2,5,3.0),(3,4,5.0),(3,5,1.0)]
G.add_weighted_edges_from(edges)

# Mostramos el grafo
layout = nx.random_layout(G,seed=10)
colors = ['red', 'green', 'lightblue', 'yellow', 'magenta', 'gray']
nx.draw_networkx(G, layout, node_color=colors)
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=labels);

In [ ]:
print("Matriz de adyacencias del grafo")
print(nx.adjacency_matrix(G).toarray())

Vamos a obtener por fuerza bruta el coste de todos los cortes.

In [ ]:
# Función de coste de Maxcut
def maxcut_coste(graph, bitstring):
    """
    Computa la función de coste del Maxcut para un grafo y un corte representado por un string de bits
    Args:
        graph: El grafo networkx
        bitstring: str
                   Un string con valores 0 o 1 especificando un corte en el grafo
    Returns:
        coste: float
               El coste del corte
    """
    # Matriz de adyacencias del grafo
    weight_matrix = nx.adjacency_matrix(graph).toarray()
    coste = 0
    for i, j in graph.edges():
        # Si los vértices están en conjuntos distintos suma el peso de la arista
        if bitstring[i] != bitstring[j]:
            coste += weight_matrix[i,j]

    return coste

In [ ]:
# Obtenemos por fuerza bruta los valores de todos los posibles cortes
num_vars = G.number_of_nodes()

#Creamos una lista de bitstrings con todos los posibles cortes
bitstrings = ['{:b}'.format(i).rjust(num_vars, '0') for i in range(2**num_vars)]
print(bitstrings)

costes = list()
for bitstring in bitstrings:
    costes.append(maxcut_coste(G, bitstring))
print(costes)

# Ordenamos los costes y los bitstrings en orden creciente de coste
costes, bitstrings = zip(*sorted(zip(costes, bitstrings)))

In [ ]:
print("Corte de mayor coste = ",bitstrings[len(bitstrings)-1], ", coste = ", costes[len(bitstrings)-1])


In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
# Muestra una gráfica con los valores
bar_plot = go.Bar(x = bitstrings, y = costes, marker=dict(color=costes, colorscale = 'plasma', colorbar=dict(title='Cut Value')))
fig = go.Figure(data=bar_plot, layout = dict(xaxis=dict(type = 'category'), width = 1500, height = 600))
fig.show()

-----------------------------------

Resolvemos el problema de optimización con QAOA

In [ ]:
# Nº de nodos del grafo y matriz de pesos
nnodes = len(G.nodes())
weight_matrix = nx.adjacency_matrix(G).toarray()

# Matriz Q
Q = -weight_matrix

# Vector c
c = np.zeros(nnodes)
for i in range(nnodes):
    for j in range(nnodes):
        c[i] += weight_matrix[i,j]

# Defino el problema
qubo_maxcut = QuadraticProgram('Maxcut como problema cuadrático')

# Variables binarias del problema
for i in range(nnodes):
    nombre = 'x_'+str(i)
    qubo_maxcut.binary_var(name = nombre)

# Especificalo como un problema de maximizacion e imprimelo
qubo_maxcut.maximize(quadratic = Q, linear = c)
print(qubo_maxcut.prettyprint())

In [ ]:
# Usamos el optimizador anterior sobre este problema
result = eigen_optimizer.solve(qubo_maxcut)
print(result)

#### Referencias

  - Farhi, E., Goldstone, J., & Gutmann, S. (2014). A quantum approximate optimization algorithm. arXiv preprint [arXiv:1411.4028](https://arxiv.org/abs/1411.4028)
  - Ejemplo definiendo el circuito: https://learning.quantum.ibm.com/tutorial/quantum-approximate-optimization-algorithm



---



---



---



<a name="vqe"></a>
# **Variational Quantum Eigensolvers (VQE)**

VQE es una generalización de QAOA para aproximar el estado base de un hamiltoniano H genérico. Se basa en el principio variacional, que establece que el mínimo valor esperado de un observable se alcanza siempre en un autovector del mismo. Es decir, que para cualquier estado arbitrario, el valor esperado de un observable es siempre mayor o igual al mínimo.


<details>
  <summary>Pulsa aquí para una explicación del principio variacional</summary>

El principio variacional permite estimar un límite superior para la energía del estado base (_ground state_) de un sistema cuántico.
    
Supongamos un estado cuántico, caracterizado por un hamiltoniano $H$, que representa la energía del sistema. Los posibles valores de energía son los autovalores $\lambda_i$ de la matriz $H$, y la energía del sistema en el estado $\vert u_i\rangle$ viene dada por el valor esperado de $H$ en ese estado:

$$
E(\vert u_i \rangle)\equiv \lambda_i = \langle H\rangle_{|u_i\rangle} = \langle u_i \vert H \vert u_i \rangle
$$


El estado base del sistema es el correspondiente al mínimo de energía. Supongamos que ese mínimo corresponde al autoestado $|u_{min}\rangle$ con autovalor $\lambda_{min}$. Para cualquier autovalor $\lambda_i$ se tiene que:

\begin{align*}
    E_{min} \equiv \lambda_{min} \le \lambda_i \equiv \langle H \rangle_{|u_i\rangle} = \langle u_i |H|u_i \rangle
\end{align*}  

donde $|u_i\rangle$ es el autoestado asociado a $\lambda_i$.

Sea un estado arbitrario $|\psi\rangle$. Escribiendo $H$ en la base de sus autoestados $H = \sum_{i=1}^N \lambda_i |u_i\rangle\langle u_i|$ tenemos que el valor esperado de la energía en el estado $|\psi\rangle$:

\begin{align}
E(|\psi\rangle)=\langle H\rangle_{|\psi\rangle} = \langle \psi |H|\psi \rangle & = \
 \langle \psi | \left( \sum_{i=1}^N \lambda_i |u_i\rangle\langle u_i| \right) |\psi \rangle\\
 &= \sum_{i=1}^N \lambda_i \langle \psi |u_i\rangle\langle u_i|\psi \rangle\\
 &= \sum_{i=1}^N \lambda_i |\langle u_i|\psi \rangle|^2
\end{align}

Es decir, el valor esperado de $H$ en un estado cualquiera es una combinación lineal en la que los autovalores actúan como pesos. De esta expresión es facil ver que:

\begin{align}
    \lambda_{min} \le \langle H \rangle_{|\psi\rangle} = \langle \psi | H | \psi \rangle = \sum_{i = 1}^{N} \lambda_i | \langle u_i | \psi\rangle |^2
\end{align}

Esa expresión se denomina _principio variacional_ y simplemente indica que el valor mínimo de energía es menor o igual que el valor esperado de $H$ en un estado arbitrario.

</details>

VQE se suele usar en problemas de química computacional, por ejemplo, para obtener el mínimo estado de energía de una determinada molécula. También se puede usar en problemas de optimización combinatoria.

VQE aproxima el estado base de un hamiltoniano H. Para ello se crea un circuito cuántico variacional (denominado _forma variacional_ o _ansatz_) en un estado inicial $|\psi\rangle$. Ejecutando el circuito, se obtiene el valor esperado $\langle H \rangle_{|\psi\rangle}$. Se usa un  optimizador clásico para ajustar los parámetros del circuito con con vistas a encontrar los parámetros que minimicen $\langle H \rangle_{|\psi\rangle}$.

<center><img src="https://drive.google.com/uc?export=view&id=1O9RXUnTfu09PLD7O4XT1FrC2GcSE008t" alt="Circuito variacional" width="900"  /></center>


La idea de VQE es, partiendo del estado inicial, ir recorriendo el espacio de estados (o lo que es lo mismo, la esfera de Bloch) y calcular el valor esperado en $H$ en cada estado.

Por ejemplo, se podría elegir como valor inicial el estado $|0\rangle$ e ir aplicando rotaciones $R_Y$ o $R_x$:

<center><img src="https://drive.google.com/uc?export=view&id=1y94qu2xOloSXoTe-8t9nwgehEvgfZHG1" alt="VQE" width="200"  /></center>
(Fuente: <a href=https://www.mustythoughts.com/variational-quantum-eigensolver-explained>https://www.mustythoughts.com/variational-quantum-eigensolver-explained</a>)

En general, se usan ansätze $V(\theta)$ más complejos que permitan recorrer la esfera de Bloch.

#### Optimización de los parámetros

El algoritmo VQE parte de un Hamiltoniano $H$, cuyo estado base resuelve el problema, y procede como sigue:

1. Se elige un estado inicial (normalmente $|\psi(\theta)\rangle = |0\rangle$) y un _ansatz_ $V(\theta)$
2. Se le dan valores iniciales a los parámetros $\theta$
2. En la QPU se ejecuta el ansatz con esos parámetros para obtener un estado $|\psi(\theta)\rangle = V(\theta)|0\rangle$
3. Se mide el valor esperado del Hamiltoniano en ese estado $\langle\psi(\theta)|H|\psi(\theta)\rangle$
4. En la CPU se usa un algoritmo de optimización para modificar los parámetros del ansatz
    - Objetivo: reducir el valor esperado del Hamiltoniano
5. Se vuelve al paso 3 hasta que se alcanza un mínimo

Igual que en QAOA, el proceso de optimización de los parámetros puede ser un simple gradiente descendente, en el que cada parámetro se actualiza en la dirección que conduzca al mayor descenso de la energía, u otros método de optimización más sofisticados.

#### Obtención del valor esperado

A diferencia del QAOA, en el que el Hamiltoniano es una matriz diagonal, en VQE es más general $\implies$ no es trivial obtener el valor esperado.

Solución: Expresar el Hamiltoniano como suma de productos tensor de matrices de Pauli y obtener el valor esperado tomando medidas en diferentes bases.

#### Ejemplos de ansätze

La elección de la forma variacional o ansatz para VQE depende del problema a tratar. Un ejemplo es _RealAmplitudes_ que vimos antes. Otra forma es la _EfficientSU2_.

In [ ]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement='linear', insert_barriers=True)

ansatz.decompose().draw('mpl')

### Obtención de estados excitados

El algoritmo VQD (_Variational Quantum Deflaction_) es una extensión del VQE que permite obtener _estados excitados_, es decir, autoestados con mayor energía.

Para ello, si $|\psi_0\rangle$ es el estado base, es posible demostrar que el estado base del siguiente Hamiltoniano:

$$
H' = H + C|\psi_0\rangle\langle\psi_0|
$$
es el primer estado excitado de H

#### Referencias:

  - Peruzzo, et al. (2014). A variational eigenvalue solver on a photonic quantum processor. Nature communications, 5(1), 1-7. [arXiv:1304.3061](https://arxiv.org/abs/1304.3061)
  - Higgott, O., Wang, D., & Brierley, S. (2019). Variational quantum computation of excited states. Quantum, 3, 156. [arXiv:1805.08138](https://arxiv.org/abs/1805.08138)
  - Combarro, E.F. & Gonzákez-Castillo, S. (2023). A Practical Guide  to Quantum Machine Learning and Quantum Optimization, capítulo  7. Packt.
  - Qiskit tutorial: Variational quantum eigensolver https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver




---



---



---

